In [ ]:
from rdflib import Graph, Dataset, URIRef, Literal, Namespace, RDF, XSD
from rdflib.namespace import OWL, RDF, RDFS
from iribaker import to_iri
import  pandas as pd
import logging
from parse import *

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
def getWDLabel(uri):
    sparq1=SPARQLWrapper("https://wikidata.demo.openlinksw.com/sparql")
    queryString= """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
        PREFIX wd: <http://www.wikidata.org/entity/> 
        SELECT  *
        WHERE {
        %s rdfs:label ?label .
        FILTER (langMatches( lang(?label), "EN" ) )
      } 
      limit 1
      """%(uri)
    sparq1.setQuery (queryString)
    sparq1.setReturnFormat(JSON)
    results=sparq1.query().convert()
    for r in results["results"]["bindings"]:
        return(r["label"]["value"])
    
    
def GetClassLabel(URIString):
    if len(URIString.split(':')) > 2:
        Label = URIString.split(':')[-1]
    else:
        Label = URIString.split('/')[-1]
    return Label
    


In [ ]:
mapping = pd.read_csv('wd2schema-std.csv')
mapping = mapping[mapping['reltype'] == 'equivClass']
mapping = mapping.reset_index()

mapping = mapping[['sch_id', 'wd_id']]
mapping['sch_cname']=mapping['sch_id']
mapping['wd_cname']= mapping['wd_id']

mapping['wd_id'] = ["http://www.wikidata.org/wiki/" + i for i in mapping['wd_id']]
mapping['sch_id'] = ["http://schema.org/" + i for i in mapping['sch_id']]

#mapping2=pd.read_csv('wd_class_name_mapping.csv', index_col=0)
mapping['sch_cname']=[GetClassLabel(i) for i in mapping['sch_id']]
#mapping2['wd_id']=mapping['wd_id']

mapping['wd_id']=[str(i.rsplit('/',1)[0])+ '/'+str(getWDLabel('wd:'+(i.rsplit('/',1)[1]))).replace(" ", "_").lower() for i in mapping['wd_id']]


mapping

In [ ]:
#Removing 2-to-1 mappings
Del=['AdministrativeArea', 'BarOrPub', 'Physiotherapy','PhysicalTherapy', 'Place', 'BodyOfWater', 'Service']
mapping=mapping[~mapping['sch_cname'].isin(Del)]
mapping = mapping.reset_index(drop=True)
mapping.to_csv('wd_class_name_mapping.csv')